In [1]:
import tensorflow as tf

import sys
sys.path.append('..')

from src.model import GoldenRetriever
gr = GoldenRetriever(**{'learning_rate':0.001, 'beta_1':0.9, 'beta_2':0.999})


timing init
model initiated!
CPU times: user 18.6 s, sys: 1.87 s, total: 20.5 s
Wall time: 35.3 s



### 1. Text KB to Vectorised KB

In [10]:
# https://biblehub.com/commentaries/expositors/genesis/1.htm
paragraphs = ['It is therefore unreasonable to allow our reverence for this writing to be lessened because it does not anticipate the discoveries of physical science; or to repudiate its authority in its own department of truth because it does not give us information which it formed no part of the writer’s object to give. As well might we deny to Shakespeare a masterly knowledge of human life, because his dramas are blotted by historical anachronisms. That the compiler of this book of Genesis did not aim at scientific accuracy in speaking of physical details is obvious, not merely from the general scope and purpose of the Biblical writers, but especially from this, that in these first two chapters of his book he lays side by side two accounts of man’s creation which no ingenuity can reconcile. These two accounts, glaringly incompatible in details, but absolutely harmonious in their leading ideas, at once warn the reader that the writer’s aim is rather to convey certain ideas regarding man’s spiritual history and his connection with God, than to describe the process of creation. He does describe the process of creation, but he describes it only for the sake of the ideas regarding man’s relation to God and God’s relation to the world which he can thereby convey. Indeed what we mean by scientific knowledge was not in all the thoughts of the people for whom this book was written. The subject of creation, of the beginning of man upon earth, was not approached from that side at all; and if we are to understand what is here written we must burst the trammels of our own modes of thought and read these chapters not as a chronological, astronomical, geological, biological statement, but as a moral or spiritual conception.',
'It will, however, be said, and with much appearance of justice, that although the first object of the writer was not to convey scientific information, yet he might have been expected to be accurate in the information he did advance regarding the physical universe. This is an enormous assumption to make on a priori grounds, but it is an assumption worth seriously considering because it brings into view a real and important difficulty which every reader of Genesis must face. It brings into view the twofold character of this account of creation. On the one hand it is irreconcilable with the teachings of science. On the other hand it is in striking contrast to the other cosmogonies which have been handed down from prescientific ages. These are the two patent features of this record of creation and both require to be accounted for. Either feature alone would be easily accounted for; but the two co-existing in the same document are more baffling. We have to account at once for a want of perfect coincidence with the teachings of science, and for a singular freedom from those errors which disfigure all other primitive accounts of the creation of the world. The one feature of the document is as patent as the other and presses equally for explanation.',
'IF anyone is in search of accurate information regarding the age of this earth, or its relation to the sun, moon, and stars, or regarding the order in which plants and animals have appeared upon it, he is referred to recent textbooks in astronomy, geology, and palaeontology. No one for a moment dreams of referring a serious student of these subjects to the Bible as a source of information. It is not the object of the writers of Scripture to impart physical instruction or to enlarge the bounds of scientific knowledge. But if any one wishes to know what connection the world has with God, if he seeks to trace back all that now is to the very fountain-head of life, if he desires to discover some unifying principle, some illuminating purpose in the history of this earth, then we confidently refer him to these and the subsequent chapters of Scripture as his safest, and indeed his only, guide to the information he seeks. Every writing must be judged by the object the writer has in view. If the object of the writer of these chapters was to convey physical information, then certainly it is imperfectly fulfilled. But if his object was to give an intelligible account of God’s relation to the world and to man, then it must be owned that he has been successful in the highest degree.']

In [26]:
def parse_sentence_context(paragraph):
    """
    Breaks the paragraph into a list of sentence, context pairs
    to be fed into USEQA
    """
    sentence_context_pair = []
    
    for sentence in paragraph.split('.'):
        sentence = sentence.strip()
        if len(sentence) > 0:
            sentence_context_pair.append([sentence, paragraph])
    return sentence_context_pair

kb = [parse_sentence_context(paragraph) for paragraph in paragraphs]
# collapse the list by one level
kb = sum(kb, [])

In [49]:
# load the text
gr.text = {'default_kb':kb}

# calculate sentence embeddings and store
vec_knowledge = [gr.predict(sentence, context) for sentence, context in kb]
gr.vectorized_knowledge['default_kb'] = tf.concat(vec_knowledge, axis=0)

### 2. Make Queries

In [53]:
gr.make_query("Is the bible scientific?", top_k=2)

([['On the one hand it is irreconcilable with the teachings of science',
   'It will, however, be said, and with much appearance of justice, that although the first object of the writer was not to convey scientific information, yet he might have been expected to be accurate in the information he did advance regarding the physical universe. This is an enormous assumption to make on a priori grounds, but it is an assumption worth seriously considering because it brings into view a real and important difficulty which every reader of Genesis must face. It brings into view the twofold character of this account of creation. On the one hand it is irreconcilable with the teachings of science. On the other hand it is in striking contrast to the other cosmogonies which have been handed down from prescientific ages. These are the two patent features of this record of creation and both require to be accounted for. Either feature alone would be easily accounted for; but the two co-existing in the s

In [54]:
gr.make_query("Is Genesis hard to read?", top_k=2)

([['This is an enormous assumption to make on a priori grounds, but it is an assumption worth seriously considering because it brings into view a real and important difficulty which every reader of Genesis must face',
   'It will, however, be said, and with much appearance of justice, that although the first object of the writer was not to convey scientific information, yet he might have been expected to be accurate in the information he did advance regarding the physical universe. This is an enormous assumption to make on a priori grounds, but it is an assumption worth seriously considering because it brings into view a real and important difficulty which every reader of Genesis must face. It brings into view the twofold character of this account of creation. On the one hand it is irreconcilable with the teachings of science. On the other hand it is in striking contrast to the other cosmogonies which have been handed down from prescientific ages. These are the two patent features of t

In [63]:
gr.make_query("What is Genesis about?", top_k=1)

([['That the compiler of this book of Genesis did not aim at scientific accuracy in speaking of physical details is obvious, not merely from the general scope and purpose of the Biblical writers, but especially from this, that in these first two chapters of his book he lays side by side two accounts of man’s creation which no ingenuity can reconcile',
   'It is therefore unreasonable to allow our reverence for this writing to be lessened because it does not anticipate the discoveries of physical science; or to repudiate its authority in its own department of truth because it does not give us information which it formed no part of the writer’s object to give. As well might we deny to Shakespeare a masterly knowledge of human life, because his dramas are blotted by historical anachronisms. That the compiler of this book of Genesis did not aim at scientific accuracy in speaking of physical details is obvious, not merely from the general scope and purpose of the Biblical writers, but espec

In [60]:
gr.make_query("How do we read Genesis?", top_k=2)

([['That the compiler of this book of Genesis did not aim at scientific accuracy in speaking of physical details is obvious, not merely from the general scope and purpose of the Biblical writers, but especially from this, that in these first two chapters of his book he lays side by side two accounts of man’s creation which no ingenuity can reconcile',
   'It is therefore unreasonable to allow our reverence for this writing to be lessened because it does not anticipate the discoveries of physical science; or to repudiate its authority in its own department of truth because it does not give us information which it formed no part of the writer’s object to give. As well might we deny to Shakespeare a masterly knowledge of human life, because his dramas are blotted by historical anachronisms. That the compiler of this book of Genesis did not aim at scientific accuracy in speaking of physical details is obvious, not merely from the general scope and purpose of the Biblical writers, but espec